In [28]:
class Person:
    pass

In [29]:
p = Person()

try:
    p.name
except  AttributeError as e:
    print(e)

'Person' object has no attribute 'name'


In [38]:
class Person:

    def __getattribute__(self, item):
        print(f" 1st Trying to find {item}")
        return super().__getattribute__(item)

    def __getattr__(self, item):
        print(f" if __getattribute__ was not able to find {item}, so it called __getattr__")
        return "Not Found"

    def hello(self):
        return "Hello"

In [39]:
p = Person()
print(p.name)


 1st Trying to find name
 if __getattribute__ was not able to find name, so it called __getattr__
Not Found


In [40]:
p.hello()

 1st Trying to find hello


'Hello'

In [41]:
p.bye()

 1st Trying to find bye
 if __getattribute__ was not able to find bye, so it called __getattr__


TypeError: 'str' object is not callable

Try to find attribute within private attributes

In [32]:
class Person:

    def __init__(self, age):
        self._age = 0
    def __getattr__(self, item):
        alt_name = "_" + item
        print(f" __getattribute__ was not able to find {item}, so it called __getattr__")
        try:
            print(f'Trying to find {alt_name}')
            return super().__getattribute__(alt_name)  # !!! call super to avoid inifinite recursion
        except AttributeError:
            raise AttributeError(f"Attribute {item} not found")

In [33]:
p = Person(10)
p.age

 __getattribute__ was not able to find age, so it called __getattr__
Trying to find _age


0

Prevent access to private variables


In [34]:
class Person:

    def __init__(self, name):
        self._name = name

    @property
    def name(self):
        return super().__getattribute__('_name')

    @name.setter
    def name(self, value):
        self._name = value

    def __getattribute__(self, item):
        if item.startswith('_') and not item.startswith('__'):
            raise AttributeError(f"Attribute {item} is private")
        return super().__getattribute__(item)

In [35]:
p = Person("John")
print(p.name)
try:
    p._name
except AttributeError as e:
    print(e)

John
Attribute _name is private


In [36]:
p.__dict__

{'_name': 'John'}

========================================================================================================

============================================  Attribute Write accessors ==================

In [43]:
class Person:
    def __setattr__(self, key, value):
        print(f'Setting {key} to {value}')
        super().__setattr__(key, value)

In [44]:
p = Person()
p.name = "John"

Setting name to John


Same with metaclass

In [47]:
class PersonMeta(type):
    def __setattr__(cls, key, value):
        print(f'Setting class attr {key} to {value}')
        super().__setattr__(key, value)

class Person(metaclass=PersonMeta):
    def __setattr__(cls, key, value):
        print(f'Setting instance attr {key} to {value}')
        super().__setattr__(key, value)

In [48]:
Person.name = "John"

Setting class attr name to John


In [49]:
p = Person()
p.name = "Val"

Setting instance attr name to Val


======================Use value descriptors

In [50]:
class DataDescriptor:
    def __init__(self, name):
        self.name = name

    def __get__(self, instance, owner):
        print(f"Descriptor Getting {self.name}")
        return instance.__dict__.get(self.name)

    def __set__(self, instance, value):
        print(f"Descriptor Setting {self.name} to {value}")
        instance.__dict__[self.name] = value

In [51]:
class NonDataDescriptor:
    def __init__(self, name):
        self.name = name

    def __get__(self, instance, owner):
        print(f"Descriptor Getting {self.name}")
        return instance.__dict__.get(self.name)


In [52]:
class MyClass:
    data = DataDescriptor("data")
    non_data = NonDataDescriptor("non_data")

    def __setattr__(self, key, value):
        print(f"Instance __setattr__ called {key} to {value}")
        super().__setattr__(key, value)

In [53]:
m = MyClass()
m.__dict__

{}

In [54]:
m.data = 10

Instance __setattr__ called data to 10
Descriptor Setting data to 10


In [55]:
m.non_data =500 # !! __setattr__ was called on instance despirte it is non-data descriptor which does not have __set__ method

Instance __setattr__ called non_data to 500


In [56]:
m.__dict__

{'data': 10, 'non_data': 500}